### FastAPI Demo

In [1]:
!pip install fastapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 1.6 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 80 kB 7.7 MB/s 


In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/kaustubhgupta/Technocolab-Final-Project/master/Data/cleaned.csv",  index_col=None)

In [4]:
df.dtypes

track_id              int64
genre_top            object
acousticness        float64
danceability        float64
energy              float64
instrumentalness    float64
liveness            float64
speechiness         float64
tempo               float64
valence             float64
dtype: object

In [5]:
df.head(10)

,track_id,genre_top,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
0,153,Rock,0.988306,0.255661,0.979774,0.973006,0.121342,0.051740,90.241,0.034018
1,154,Rock,0.970135,0.352946,0.023852,0.957113,0.113261,0.032177,53.758,0.035632
2,155,Rock,0.981657,0.142249,0.912122,0.967294,0.363510,0.087527,91.912,0.034325
3,169,Rock,0.989141,0.225978,0.722835,0.263076,0.092371,0.053406,94.322,0.028347
4,170,Rock,0.886660,0.298518,0.744333,0.920950,0.139587,0.088781,97.880,0.073548
5,171,Rock,0.698278,0.285816,0.213494,0.955691,0.087036,0.064094,125.645,0.150599
6,172,Rock,0.815549,0.144125,0.892721,0.900430,0.104703,0.102294,138.680,0.034916
7,173,Rock,0.842113,0.285293,0.564689,0.951624,0.110481,0.040611,166.552,0.254299
8,174,Rock,0.367393,0.339698,0.846639,0.909186,0.198944,0.462607,70.887,0.048243
9,175,Rock,0.993928,0.310090,0.952493,0.274311,0.354231,0.825744,69.413,0.033402


In [6]:
X = df.drop(['track_id', 'genre_top'], axis=1)
y = df.genre_top
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y)

tree = DecisionTreeClassifier(random_state=42)

tree.fit(X_train, y_train)

import pickle

pickle_file = "model_tree.pkl"  

with open(pickle_file, 'wb') as file:  
    pickle.dump(tree, file)

In [7]:
from pydantic import BaseModel

class Music(BaseModel):
    acousticness: float 
    danceability: float 
    energy: float 
    instrumentalness: float 
    liveness: float 
    speechiness: float 
    tempo: float 
    valence: float
    class Config:
        schema_extra = {
            "example": {
                "acousticness": 0.838816, 
                "danceability": 0.542950, 
                "energy": 0.669215,
                "instrumentalness": 0.000006,
                "liveness": 0.105610,
                "speechiness": 0.391221,
                "tempo": 111.894,
                "valence": 0.796073
            }
        }

In [8]:
from fastapi import FastAPI
import pickle

app = FastAPI()

@app.on_event("startup")
def load_model():
    global model
    model = pickle.load(open("model_tree.pkl", "rb"))

@app.get('/')
def index():
    return {'message': 'This is the homepage of the API '}


@app.post('/predict')
def get_music_category(data: Music):
    received = data.dict()
    acousticness = received['acousticness']
    danceability = received['danceability']
    energy = received['energy']
    instrumentalness = received['instrumentalness']
    liveness = received['liveness']
    speechiness = received['speechiness']
    tempo = received['tempo']
    valence = received['valence']
    pred_name = model.predict([[acousticness, danceability, energy,
                                instrumentalness, liveness, speechiness, tempo, valence]]).tolist()[0]
    return {'prediction': pred_name}

In [9]:
!pip install colabcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 2.7 MB/s 
     |████████████████████████████████| 45 kB 3.2 MB/s 
     |████████████████████████████████| 8.3 MB 51.6 MB/s 
     |████████████████████████████████| 423 kB 49.5 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 9.7 MB 41.9 MB/s 
     |████████████████████████████████| 344 kB 70.5 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
     |████████████████████████████████| 561 kB 69.6 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 132 kB 49.8 MB/s 
     |████████████████████████████████| 131 kB 55.9 MB/s 
     |████████████████████████████████| 131 kB 55.7 MB/s 
     |████████████████████████████████| 130 kB 56.0 MB/s 
     |████████████████████████████████| 130 kB 50.5 MB/s 
     |████████████████████████████████| 

In [11]:
from colabcode import ColabCode
server = ColabCode(port=2000, code=False)
server.run_app(app=app)

INFO:     Started server process [58]
INFO:uvicorn.error:Started server process [58]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:2000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:2000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://64ec-34-139-195-119.ngrok.io" -> "http://localhost:2000"
INFO:     2409:4072:796:4faa:d97:dfcf:1ead:4fea:0 - "GET / HTTP/1.1" 200 OK
INFO:     2409:4072:796:4faa:d97:dfcf:1ead:4fea:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2409:4072:796:4faa:d97:dfcf:1ead:4fea:0 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [58]
INFO:uvicorn.error:Finished server process [58]
